In [ ]:
import os
path = "/Users/patricia/Documents/code/python-code/behavior-detection/src"
os.chdir(path)  # Muda o diretório para o nível anterior (a raiz do projeto)
print(os.getcwd())  # Verifique se agora está na raiz

# Load data

In [ ]:
from behavior.data.behavior_data_loader import BehaviorDataLoader

data_path = '../data/new_logs_labels.csv'

data = BehaviorDataLoader.load_data(data_path, delimiter=';')
print(data.shape)
data.head(5)

In [ ]:
from core.preprocessors.data_cleaner import DataCleaner

print("Valores da coluna 'comportamento' antes da remoção:", data['comportamento'].value_counts())

# Remove instances where 'comportamento' is '?'
data = DataCleaner.remove_instances_with_value(data, 'comportamento', '?')

print("\nValores da coluna 'comportamento' depois da remoção:", data['comportamento'].value_counts())

In [ ]:
data.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

# Select a subset of the data only for testing purposes

print("Tamanho do dataframe antes:", data.shape)
data, _ = train_test_split(data, test_size=0.2, stratify=data['comportamento'], random_state=42)
data.reset_index(drop=True, inplace=True)
print("Tamanho do dataframe após:", data.shape)

# Pre-processing

## Remove unnecessary columns

In [ ]:
# Removing columns related to IDs, emotions, personality and behaviors, because 
# we want to classify behaviors only by the students' interactions with the system
columns_to_remove_ids = ['id_log', 'grupo', 'num_dia', 'num_log']
columns_to_remove_emotions = [
    'estado_afetivo', 'estado_engajamento_concentrado', 
    'estado_confusao', 'estado_frustracao', 'estado_tedio', 'estado_indefinido', 
    'ultimo_estado_afetivo', 'ultimo_engajamento_concentrado', 'ultimo_confusao', 
    'ultimo_frustracao', 'ultimo_tedio', 'ultimo_estado_indefinido'
]
columns_to_remove_personality = [
    'traco_amabilidade_fator', 'traco_extrovercao_fator', 'traco_conscienciosidade_fator', 
    'traco_abertura_fator', 'traco_neuroticismo_fator', 'traco_amabilidade_cat', 
    'traco_extrovercao_cat', 'traco_conscienciosidade_cat', 'traco_abertura_cat', 
    'traco_neuroticismo_cat']

columns_to_remove_behaviors = [
    'comportamento_on_task', 'comportamento_on_task_conversation', 'comportamento_on_task_out',
    'comportamento_off_task', 'comportamento_on_system', 'comportamento_indefinido',
    'ultimo_comportamento', 'ultimo_comportamento_on_task', 'ultimo_comportamento_on_task_conversation',
    'ultimo_comportamento_on_task_out', 'ultimo_comportamento_off_task', 'ultimo_comportamento_on_system',
    'ultimo_comportamento_indefinido'
]

columns_to_remove = columns_to_remove_ids + \
        columns_to_remove_emotions + \
        columns_to_remove_personality + \
        columns_to_remove_behaviors

cleaned_data = DataCleaner.remove_columns(data, columns_to_remove)


In [ ]:
cleaned_data.head(5)

In [ ]:
# Preenche valores ausentes no DataFrame X com a string 'missing'.

numeric_columns = cleaned_data.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = cleaned_data.select_dtypes(exclude=['float64', 'int64']).columns

cleaned_data[numeric_columns] = cleaned_data[numeric_columns].fillna(cleaned_data[numeric_columns].median())
cleaned_data[categorical_columns] = cleaned_data[categorical_columns].fillna('missing')


## Split data by student level into training and test datasets

In [ ]:
from core.preprocessors.data_splitter import DataSplitter

train_data, test_data = DataSplitter.split_by_student_level(cleaned_data, test_size=0.2, column_name='aluno')

In [ ]:
# 2. Após o split por nível do estudante
print("\n=== Após split por nível do estudante ===")
print(f"Shape de train_data: {train_data.shape}")
print("Colunas em train_data:", train_data.columns.tolist())

In [ ]:
# removing the 'aluno' column from the data after splitting into train and test sets

# Remover 'aluno' do conjunto de treinamento
train_data = DataCleaner.remove_columns(train_data, ['aluno'])

# Remover 'aluno' do conjunto de teste
test_data = DataCleaner.remove_columns(test_data, ['aluno'])

In [ ]:
# 1. Após remover coluna 'aluno'
print("\n1. Após remover 'aluno':")
print(f"Shape de train_data: {train_data.shape}")

## Split data into Features (X) and Target (y)

In [ ]:
from core.preprocessors.data_splitter import DataSplitter

# Conjunto de treinamento
X_train, y_train = DataSplitter.split_into_x_y(train_data, 'comportamento')

# Conjunto de teste
X_test, y_test = DataSplitter.split_into_x_y(test_data, 'comportamento')

In [ ]:
import pandas as pd
import numpy as np
# 2. Após split X/y
print("\n2. Após split X/y:")
if isinstance(X_train, pd.DataFrame):
    print(f"Shape de X_train: {X_train.shape}")
    print("Primeiras colunas de X_train:", list(X_train.columns)[:5])
else:
    print("X_train não é um DataFrame!")
    print(f"Tipo de X_train: {type(X_train)}")

In [ ]:
print("Primeiras 5 instâncias de y_train:")
print(y_train[:5])

print("\nPrimeiras 5 instâncias de y_test:")
print(y_test[:5])

## Encoding variables

### Encoding true labels (y)

In [ ]:
import importlib
from core.preprocessors import column_selector, data_encoder
from behavior.data import behavior_data_encoder

# Recarregar o módulo para garantir que as alterações sejam aplicadas
importlib.reload(column_selector)
importlib.reload(data_encoder)
importlib.reload(behavior_data_encoder)

In [ ]:
# Encoding y_train and y_test
from behavior.data.behavior_data_encoder import BehaviorDataEncoder

# Codificar y_train
y_train = BehaviorDataEncoder.encode_y(y_train)

# Codificar y_test
y_test = BehaviorDataEncoder.encode_y(y_test)



### Encoding features (X)

In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
from behavior.data.behavior_data_encoder import BehaviorDataEncoder

# Encoding do target (y)
y_train = BehaviorDataEncoder.encode_y(y_train)
y_test = BehaviorDataEncoder.encode_y(y_test)

# Encoding das features (X)
print("=== Iniciando encoding das features ===")
X_encoder = BehaviorDataEncoder(num_classes=5)
print("\nRealizando fit do encoder...")
X_encoder.fit(X_train)

print("\nRealizando transform...")
X_train = X_encoder.transform(X_train)

print("\nTransformando dados de teste...")
X_test = X_encoder.transform(X_test)

# Verificação final
print("\n=== Verificação após encoding ===")
print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de X_test: {X_test.shape}")
print(f"Shape de y_train: {y_train.shape}")
print(f"Shape de y_test: {y_test.shape}")

In [ ]:
print(X_test.shape)
print(X_test.head(10))

In [ ]:
# Antes do SMOTE, adicione estas verificações
print("Verificando X_train antes do SMOTE:")
print("1. Shape de X_train:", X_train.shape)
print("2. Tipo de X_train:", type(X_train))
print("3. Shape de y_train:", y_train.shape)
print("4. Tipo de y_train:", type(y_train))

if isinstance(X_train, pd.DataFrame):
    print("5. Colunas em X_train:")
    print(X_train.columns.tolist())
    print("\n6. Primeiras linhas de X_train:")
    print(X_train.head())
    print("\n7. Tipos de dados das colunas:")
    print(X_train.dtypes)

# Balanceamento dos dados

In [ ]:
from core.preprocessors.data_balancer import DataBalancer

data_balancer = DataBalancer()
X_train, y_train = data_balancer.apply_smote(X_train, y_train)

In [ ]:
from collections import Counter

print(f"Resampled dataset shape: {Counter(y_train)}")

# Treinamento dos Modelos

## Definindo parametros

In [ ]:
# Importações e configuração de diretório permanecem iguais até a seção de treinamento

# Na seção "Definindo parametros", substituir:
from core.models.multiclass.behavior_model_params import BehaviorModelParams

# Criar instância dos parâmetros específicos para comportamentos
model_params = BehaviorModelParams()

# # Definir quais modelos e seletores utilizar
# selected_models = [ 
#     # 'Logistic Regression',
#     'Decision Tree',
#     # 'Random Forest',
#     # 'Gradient Boosting',
#     # 'SVM',
#     # 'KNN',
#     # 'XGBoost',
#     'Naive Bayes' 
#     # 'MLP'  
# ]

# # Definir quais seletores de features utilizar
# selected_selectors = [
#     # 'rfe',      # Recursive Feature Elimination
#     'pca',      # Principal Component Analysis
#     # 'rf',       # Random Forest Feature Selector
#     # 'mi',       # Mutual Information Feature Selector
#     'none'      # Sem seleção de features
# ]


# # Usar todos os modelos disponíveis
selected_models = model_params.get_available_models()  # ou lista específica

# # Usar todos os seletores disponíveis
selected_selectors = None  # None to use all selectors

# Configurar validação cruzada estratificada
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Parâmetros de otimização
n_iter = 50  # Reduzido para teste inicial
n_jobs = 6  # MacBook Air M2 tem 8 núclos CPUs e 10 GPUs. Como uso sciktlearn, só posso usar CPUs. Teria que usar Pytorch ou TensorFlow para usar GPUs
scoring_metric = 'balanced_accuracy'



## Usando Otimização Bayesiana (Optuna)

In [ ]:
from core.management.stage_training_manager import StageTrainingManager
from core.training.optuna_bayesian_optimization_training import OptunaBayesianOptimizationTraining

# Definir as etapas incorporando todos os seletores de features
stages = [
        ('etapa_1_logistic_none', ['Logistic Regression'], ['none']),
        ('etapa_2_logistic_pca', ['Logistic Regression'], ['pca']),
        ('etapa_3_logistic_rfe', ['Logistic Regression'], ['rfe']),
        ('etapa_4_logistic_rf', ['Logistic Regression'], ['rf']),
        ('etapa_5_logistic_mi', ['Logistic Regression'], ['mi']),
        ('etapa_6_tree_none', ['Decision Tree'], ['none']),
        ('etapa_7_tree_pca', ['Decision Tree'], ['pca']),
        ('etapa_8_tree_rfe', ['Decision Tree'], ['rfe']),
        ('etapa_9_tree_rf', ['Decision Tree'], ['rf']),
        ('etapa_10_tree_mi', ['Decision Tree'], ['mi']),
        ('etapa_11_rf_none', ['Random Forest'], ['none']),
        ('etapa_12_rf_pca', ['Random Forest'], ['pca']),
        ('etapa_13_rf_rfe', ['Random Forest'], ['rfe']),
        ('etapa_14_rf_rf', ['Random Forest'], ['rf']),
        ('etapa_15_rf_mi', ['Random Forest'], ['mi']),
        ('etapa_16_gb_none', ['Gradient Boosting'], ['none']),
        ('etapa_17_gb_pca', ['Gradient Boosting'], ['pca']),
        ('etapa_18_gb_rfe', ['Gradient Boosting'], ['rfe']),
        ('etapa_19_gb_rf', ['Gradient Boosting'], ['rf']),
        ('etapa_20_gb_mi', ['Gradient Boosting'], ['mi']),
        ('etapa_21_svm_none', ['SVM'], ['none']),
        ('etapa_22_svm_pca', ['SVM'], ['pca']),
        ('etapa_23_svm_rfe', ['SVM'], ['rfe']),
        ('etapa_24_svm_rf', ['SVM'], ['rf']),
        ('etapa_25_svm_mi', ['SVM'], ['mi']),
        ('etapa_26_knn_none', ['KNN'], ['none']),
        ('etapa_27_knn_pca', ['KNN'], ['pca']),
        ('etapa_28_knn_rfe', ['KNN'], ['rfe']),
        ('etapa_29_knn_rf', ['KNN'], ['rf']),
        ('etapa_30_knn_mi', ['KNN'], ['mi']),
        ('etapa_31_xgb_none', ['XGBoost'], ['none']),
        ('etapa_32_xgb_pca', ['XGBoost'], ['pca']),
        ('etapa_33_xgb_rfe', ['XGBoost'], ['rfe']),
        ('etapa_34_xgb_rf', ['XGBoost'], ['rf']),
        ('etapa_35_xgb_mi', ['XGBoost'], ['mi']),
        ('etapa_36_nb_none', ['Naive Bayes'], ['none']),
        ('etapa_37_nb_pca', ['Naive Bayes'], ['pca']),
        ('etapa_38_nb_rfe', ['Naive Bayes'], ['rfe']),
        ('etapa_39_nb_rf', ['Naive Bayes'], ['rf']),
        ('etapa_40_nb_mi', ['Naive Bayes'], ['mi']),
        ('etapa_41_mlp_none', ['MLP'], ['none']),
        ('etapa_42_mlp_pca', ['MLP'], ['pca']),
        ('etapa_43_mlp_rfe', ['MLP'], ['rfe']),
        ('etapa_44_mlp_rf', ['MLP'], ['rf']),
        ('etapa_45_mlp_mi', ['MLP'], ['mi'])
    ]

# Inicializar o gerenciador de treinamento
training_manager = StageTrainingManager(
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    model_params=model_params,
    n_iter=50,
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

In [ ]:
try:
    training_manager.execute_all_stages(training_manager, stages)
except Exception as e:
    print(f"\nExecução interrompida: {str(e)}")
    print("Você pode executar novamente o mesmo código para retomar do último stage não completado.")

# Avaliação e logging

In [ ]:
from core.reporting import metrics_reporter

# Após todas as etapas estarem concluídas:
final_results = training_manager.combine_results()
training_results, class_metrics, avg_metrics = final_results
metrics_reporter.generate_reports(class_metrics, avg_metrics, filename_prefix="_Final_Combined_")